# MNIST with CNN + Softmax

In [ ]:
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np



# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)

# Perform PCA to reduce input dimension from 784 to 70
pca = PCA(n_components=70)
x_train = pca.fit_transform(x_train.reshape(-1, 784))
x_test = pca.transform(x_test.reshape(-1, 784))

# Define fully connected model with two hidden layers
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_dim=70),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=1, batch_size=200, verbose=1)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', test_acc)

300/300 [==============================] - 5s 13ms/step - loss: 3.5563 - accuracy: 0.9168
Test accuracy: 0.9538999795913696


In [ ]:
"model_nsit_softmax"

# Save the model
model.save('model_nsit_softmax.h5')

# MNIST with CNN + SVM



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define the L2-SVM loss function
class L2SVM(nn.Module):
    def __init__(self, C):
        super(L2SVM, self).__init__()
        self.C = C

    def forward(self, input, target):
        scores = input
        correct_scores = scores.gather(1, target.view(-1, 1)).squeeze()
        margins = torch.clamp(scores - correct_scores.view(-1, 1) + 1, min=0)
        margins[target == -1] = 0
        loss = margins.sum() * self.C
        return loss

# Define the fully connected MLP
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x


# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

# # Reduce input data to 70 dimensions using PCA
# n_components=70
# pca = PCA(n_components=n_components)
# train_data = pca.fit_transform(train_dataset.data.view(-1, 784))
# test_data = pca.transform(test_dataset.data.view(-1, 784))

# # Convert numpy arrays to tensors
# train_data = torch.Tensor(train_data)
# test_data = torch.Tensor(test_data)
# train_labels = train_dataset.targets
# test_labels = test_dataset.targets

# # Create data loaders
# train_dataset = torch.utils.data.TensorDataset(train_data, train_labels)
# test_dataset = torch.utils.data.TensorDataset(test_data, test_labels)
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

# Initialize the model and loss function
model = MLP()
criterion = L2SVM(C=0.1)

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
num_epochs = 1

for epoch in range(num_epochs):
    train_loss = 0
    train_acc = 0
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        train_acc += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)

    print('Epoch: {} \t Train Loss: {:.6f} \t Train Acc: {:.6f}'.format(epoch+1, train_loss, train_acc))

Epoch: 1 	 Train Loss: 0.155415 	 Train Acc: 0.894617


In [ ]:
# Test the model
test_loss = 0
test_acc = 0
model.eval()

with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        loss = criterion(output, target)

        test_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        test_acc += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
test_acc /= len(test_loader.dataset)

print('Test Loss: {:.6f} \t Test Acc: {:.6f}'.format(test_loss, test_acc))

Test Loss: 0.117301 	 Test Acc: 0.956400


In [ ]:
# Save the trained model
PATH = 'model_nsit_sxm.pt'
torch.save(model.state_dict(), PATH)

In [ ]:
# Load the saved model
model = MLP()
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>